In [ ]:
%matplotlib widget

In [ ]:
import sys
sys.path.append("..")

import pandas as pd
from pathlib import Path
from tqdm import tqdm

import json
import numpy as np
import os

from data.loaders import load_capas_from_jsons, get_train_test_data

In [ ]:
base_dir = r"C:\Users\stav\data\whodis\parsed\CAPAs"
traintest_dir = r"C:\Users\stav\data\whodis\train_test_split"
output_train_val_split_dir = traintest_dir

cat_dfs= load_capas_from_jsons(base_dir)
capas_df = cat_dfs['capas']
mbcs_df = cat_dfs['mbc']

In [ ]:
cats_df = capas_df
features_df = pd.pivot_table(cats_df, values='matches', index=['uid'], columns=['rule'], aggfunc=np.sum, fill_value=0)
features_df = (features_df>=1).astype(int)
features_df['label'] = capas_df.groupby('uid').label.first()

# Split v2: Remove duplicate CAPAs

In [ ]:
trainval_ratio = 0.7
val_split = pd.DataFrame(columns=['family','sample_name'])
train_split = pd.DataFrame(columns=['family','sample_name'])
for label, group_df in features_df.drop_duplicates().groupby('label'):
    n_train = int(group_df.shape[0]*trainval_ratio)
    group_train_df = pd.DataFrame()
    group_train_df['sample_name'] = group_df.iloc[:n_train].index.to_list()
    group_train_df['family'] = label
    group_val_df = pd.DataFrame()
    group_val_df['sample_name'] = group_df.iloc[n_train:].index.to_list()
    group_val_df['family'] = label
    val_split = val_split.append(group_val_df)
    train_split = train_split.append(group_train_df)

In [ ]:
val_split.to_csv(os.path.join(output_train_val_split,'test_samples_v2.csv'), index=False)
train_split.to_csv(os.path.join(output_train_val_split,'train_samples_v2.csv'), index=False)

# Split v3: Split "Hard test samples" fro regular train and test samples (wipbot)

In [ ]:
wipbot_samples, = get_train_test_data(traintest_dir, 'wipbot')
capas_df = capas_df.set_index('uid')
capas_df.loc[wipbot_samples.index,'label']='wipbot'
capas_df = capas_df.reset_index()
features_df = pd.pivot_table(capas_df, values='matches', index=['uid'], columns=['rule'], aggfunc=np.sum, fill_value=0)
features_df = (features_df>=1).astype(int)
features_df['label'] = capas_df.groupby('uid').label.first()

In [ ]:
trainval_ratio = 0.7
hard_test_labels=['wipbot']

val_split = pd.DataFrame(columns=['family','sample_name'])
train_split = pd.DataFrame(columns=['family','sample_name'])
hard_test_split = pd.DataFrame(columns=['family','sample_name'])
for label, group_df in features_df.drop_duplicates().groupby('label'):
    print(label)
    if label in hard_test_labels:
        group_hard_df = pd.DataFrame()
        group_hard_df['sample_name'] = group_df.index.to_list()
        group_hard_df['family'] = label
        hard_test_split=hard_test_split.append(group_hard_df)
    else:
        n_train = int(group_df.shape[0]*trainval_ratio)
        group_train_df = pd.DataFrame()
        group_train_df['sample_name'] = group_df.iloc[:n_train].index.to_list()
        group_train_df['family'] = label
        group_val_df = pd.DataFrame()
        group_val_df['sample_name'] = group_df.iloc[n_train:].index.to_list()
        group_val_df['family'] = label
        val_split = val_split.append(group_val_df)
        train_split = train_split.append(group_train_df)

In [ ]:
val_split.to_csv(os.path.join(output_train_val_split,'test_samples_v3.csv'), index=False)
train_split.to_csv(os.path.join(output_train_val_split,'train_samples_v3.csv'), index=False)
hard_test_split.to_csv(os.path.join(output_train_val_split,'hard_samples_v3.csv'), index=False)